# How to connect Langchain to Oracle 26ai

!python -m pip install -U langchain-oracledb langchain-huggingface

In [ ]:
from langchain_oracledb.vectorstores import oraclevs
from langchain_oracledb.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
import oracledb

connection = oracledb.connect(
    user="ORACLE USERNAME", 
    password="ORACLE PASSWORD", 
    dsn="sample_low",
    config_dir="FOLDER CONTAINING WALLET DATA",
    wallet_location="FOLDER CONTAINING WALLET DATA",
    wallet_password="WALLET PASSWORD",
)

In [ ]:
documents_json_list = [
    {
        "id": "creating-flows",
        "text": "Metaflow follows the dataflow paradigm which models a program as a directed graph of operations. This is a natural paradigm for expressing data processing pipelines, machine learning in particular. We call the graph of operations a flow. You define the operations, called steps, which are nodes of the graph and contain transitions to the next steps, which serve as edges. Metaflow sets some constraints on the structure of the graph. For starters, every flow needs a step called start and a step called end. An execution of the flow, which we call a run, starts at start. The run is successful if the final end step finishes successfully. What happens between start and end is up to you. You can construct the graph in between using an arbitrary combination of the following three types of transitions supported by Metaflow:",
        "link": "https://docs.metaflow.org/metaflow/basics",
    },
    {
        "id": "authoring-flows",
        "text": "With Metaflow, you might start with a simple stub, perhaps just a step to load data, and then gradually add more @steps, say, for data transformation, model training, and beyond, testing the flow at each iteration. To enable a smooth development experience, these iterations should run quickly, with minimal waiting - much like the familiar workflow in a notebook, where you build results one cell at a time.",
        "link": "https://docs.metaflow.org/metaflow/authoring-flows/introduction",
    },
    {
        "id": "running-flows",
        "text": "The Runner API allows you to start and manage Metaflow runs and other operations programmatically, for instance, to run flows in a script. The Runner class exposes a blocking API, which waits for operations to complete, as well as a non-blocking (asynchronous) APIs, prefixed with async which execute operations in the background. This document provides an overview of common patterns. For detailed API documentation, see the Runner API reference.",
        "link": "https://docs.metaflow.org/metaflow/managing-flows/runner",
    }
]

In [ ]:
documents = []

for doc in documents_json_list:
    metadata = {"id": doc["id"], "link": doc["link"]}
    
    document = Document(
        page_content=doc["text"], 
        metadata=metadata
    )

    documents.append(document)

In [ ]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
vector_store = OracleVS.from_documents(
    documents,
    model,
    client=connection,
    table_name="Documents_COSINE",
    distance_strategy=DistanceStrategy.COSINE,
)

In [ ]:
oraclevs.create_index(
    connection,
    vector_store,
    params={
        "idx_name": "hnsw_idx2",
        "idx_type": "HNSW",
        "accuracy": 97,
        "parallel": 16,
    },
)

In [ ]:
query = "What is exposed by the Runner API?"
result = vector_store.similarity_search(query, 2)
print(result)